<a href="https://colab.research.google.com/github/marcosalfonzo/API-productos-SQL/blob/main/ETL__Pr%C3%A1ctico___FakeStore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTACIÓN DE LIBRERÍAS
# pandas: para manejar datos tabulares (DataFrames)
# requests: para realizar peticiones HTTP a la API
# sqlite3: para crear y manipular una base de datos SQLite

In [45]:
import pandas as pd
import requests
import sqlite3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [44]:
# URL de la API Fake Store (una API pública de prueba con productos de una tienda falsa)
url = "https://fakestoreapi.com/products"

In [16]:
# FUNCIÓN 1: Obtener datos de productos desde la API
def get_productos():
    """
    Realiza una petición GET a la Fake Store API para obtener la lista de productos.
    Convierte la respuesta JSON en un DataFrame de pandas.
    Incluye manejo de errores en caso de fallas de conexión o respuesta.

    Returns:
        pd.DataFrame: DataFrame con los productos o DataFrame vacío si hay error.
    """

    try:

         response= requests.get(url, timeout= 10)

         response.raise_for_status()

         data = response.json()

         return pd.DataFrame(data)

    except requests.exceptions.RequestException as e:

         print(f"Error al obtener los productos: {e}")

         return pd.DataFrame()


In [17]:
# Ejecución de la función
dtf = get_productos()

In [18]:
# visualización de las primeras filas
dtf.head()

,id,title,price,description,category,image,rating
0,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",109.95,Your perfect pack for everyday use and walks i...,men's clothing,https://fakestoreapi.com/img/81fPKd-2AYL._AC_S...,"{'rate': 3.9, 'count': 120}"
1,2,Mens Casual Premium Slim Fit T-Shirts,22.30,"Slim-fitting style, contrast raglan long sleev...",men's clothing,https://fakestoreapi.com/img/71-3HjGNDUL._AC_S...,"{'rate': 4.1, 'count': 259}"
2,3,Mens Cotton Jacket,55.99,great outerwear jackets for Spring/Autumn/Wint...,men's clothing,https://fakestoreapi.com/img/71li-ujtlUL._AC_U...,"{'rate': 4.7, 'count': 500}"
3,4,Mens Casual Slim Fit,15.99,The color could be slightly different between ...,men's clothing,https://fakestoreapi.com/img/71YXzeOuslL._AC_U...,"{'rate': 2.1, 'count': 430}"
4,5,John Hardy Women's Legends Naga Gold & Silver ...,695.00,"From our Legends Collection, the Naga was insp...",jewelery,https://fakestoreapi.com/img/71pWzhdJNwL._AC_U...,"{'rate': 4.6, 'count': 400}"


In [21]:
# FUNCIÓN 2: Transformar y limpiar el DataFrame
def transform_dtf(df):
    """
    La columna 'rating' contiene un diccionario anidado con 'rate' y 'count'.
    Esta función lo desanida (normaliza) y lo convierte en columnas separadas.

    Args:
        df (pd.DataFrame): DataFrame original con columna 'rating' anidada.

    Returns:
        pd.DataFrame: DataFrame limpio con columnas 'rating_rate' y 'rating_count'.
    """
    rating_df = pd.json_normalize(df['rating'])

    rating_df.columns = ['rating_rate','rating_count']

    df= df.drop(columns=["rating"])

    df_final= pd.concat([df, rating_df], axis=1)

    return df_final


In [22]:
# Aplicamos la transformación
dtf_limpio = transform_dtf(dtf)

In [24]:
dtf_limpio.head()

,id,title,price,description,category,image,rating_rate,rating_count
0,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",109.95,Your perfect pack for everyday use and walks i...,men's clothing,https://fakestoreapi.com/img/81fPKd-2AYL._AC_S...,3.9,120
1,2,Mens Casual Premium Slim Fit T-Shirts,22.30,"Slim-fitting style, contrast raglan long sleev...",men's clothing,https://fakestoreapi.com/img/71-3HjGNDUL._AC_S...,4.1,259
2,3,Mens Cotton Jacket,55.99,great outerwear jackets for Spring/Autumn/Wint...,men's clothing,https://fakestoreapi.com/img/71li-ujtlUL._AC_U...,4.7,500
3,4,Mens Casual Slim Fit,15.99,The color could be slightly different between ...,men's clothing,https://fakestoreapi.com/img/71YXzeOuslL._AC_U...,2.1,430
4,5,John Hardy Women's Legends Naga Gold & Silver ...,695.00,"From our Legends Collection, the Naga was insp...",jewelery,https://fakestoreapi.com/img/71pWzhdJNwL._AC_U...,4.6,400


In [31]:
# CREACIÓN Y CARGA DE LA BASE DE DATOS SQLITE
# Conectamos (o creamos) la base de datos local 'store.db'

base_datos = sqlite3.connect("store.db")

In [32]:
# Guardamos el DataFrame limpio en una tabla llamada 'productos'
# if_exists="replace" sobrescribe la tabla si ya existe
dtf_limpio.to_sql("productos",base_datos,if_exists="replace", index=False)

20

In [ ]:
# Confirmamos los cambios y cerramos la conexión

In [33]:
base_datos.commit()
base_datos.close()
print("Los datos se guardaron en mi base de datos Store")

In [ ]:
# CONSULTAS DE VERIFICACIÓN EN LA BASE DE DATOS
# Abrimos nuevamente la conexión para hacer consultas

In [40]:
base_datos = sqlite3.connect("store.db")

Total productos:    COUNT(*)
0        20


In [ ]:
# 1. Contamos el total de productos almacenados
print("Total productos:", pd.read_sql("SELECT COUNT(*) FROM productos", base_datos))


In [41]:
# 2. Contamos cuántos productos hay por categoría
pd.read_sql("SELECT category, COUNT(*) FROM productos GROUP BY category", base_datos)

,category,COUNT(*)
0,electronics,6
1,jewelery,4
2,men's clothing,4
3,women's clothing,6


In [42]:
# Cerramos la conexión
base_datos.close()
print("base de datos cerrada")